This program purpose is to scrap job listing for specific locations and specific job titles or categories.
We will scrap job listing from linkedin


In [7]:
"""
Installing the packages we need
"""
%pip install beautifulsoup4
%pip install requests
%pip install pandas
%pip install fpdf2

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.8 MB/s eta 0:00:00
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached fonttools-4.56.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (101 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 16.5 MB/s eta 0:00:00
Using cached fonttools-4.56.0-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.9 MB)
Using cached pillow-11.1.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)
Using cached defusedxml-0.7.1-py2.py3-none-any.whl (25 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import os
import re
from fpdf import FPDF
"""
Modify parameters as needed
"""
number_of_pages = 1
keywords =  ['engineer','healthcare','finance']
location = 'US'
"""

"""

job_post_list = []
counter = 0
job_id_list = []

for keyword in keywords:
    for i in range(0, number_of_pages):
        """
        Getting list of job posting by searching for <li> element
        """
        counter += 25
        sample_job_listing_url = f"https://www.linkedin.com/jobs/search?keywords={keyword}&location={location}&start={counter}"
        current_page_listing_url = sample_job_listing_url
        response = requests.get(sample_job_listing_url)
        list_data = response.text
        # Parsing the HTML response
        list_soup = BeautifulSoup(list_data, 'html.parser')
        page_jobs = list_soup.find_all('li')
        print(f"Page {i+1} has {len(page_jobs)} jobs")
        print(
            f'Fetching jobs, start: {counter} for url: {current_page_listing_url}')
        # Extracting information from each job
        for job in page_jobs:
            """
            For each <li> we get the job id
            Example job post data-entity-urn="urn:li:jobPosting:4179181538"
            """
            base_card_div = job.find("div", {"class": "base-card"})
            if base_card_div is None:
                continue
            # This splitting might not work if they change the format of the job_id
            job_id = base_card_div.get('data-entity-urn').split(":")[3]
            job_id_list.append({"job_id": job_id, "keyword": keyword})
            """
            Now that we have the job ids , we are going to fetch the job details
            Sample URL: https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/4179518973
            """
        print(f"job_id_list length: {len(job_id_list)}")
        """
        Once we get all the job ids, we start fetching the job details
        """
print(f"job id list: {job_id_list}")
for jobs in job_id_list:
        job_id = jobs['job_id']
        print(f"Fetching job details for job_id: {job_id}")
        job_details_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
        print(f"Fetch url: {job_details_url}")
        response = requests.get(job_details_url)
        job_soup = BeautifulSoup(response.text, 'html.parser')
        job_post = {}
        # Getting company name
        try:
            company_name = job_soup.find(
                "a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
        except:
            company_name = None
            continue
        job_post['company_name'] = company_name
        # Getting position name
        try:
            position_name = job_soup.find(
                "h2", {"class": "topcard__title"}).text.strip()
        except:
            position_name = ''
            continue
        job_post['position_name'] = position_name
        # Getting published date
        try:
            posted_time_ago = job_soup.find(
                "span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
        except:
            posted_time_ago = ''
        job_post['posted_time_ago'] = posted_time_ago
        job_post['web scrapped date'] = datetime.now().strftime(
            "%Y-%m-%d %H:%M:%S")
        # Getting number of applicants
        try:
            num_applicants = job_soup.find(
                "figcaption", {"class": "num-applicants__caption"}).text.strip()
        except:
            num_applicants = ''
        job_post['num_applicants'] = num_applicants

        # Getting job full description html
        try:
            job_description_html = job_soup.find(
                "div", {"class": "decorated-job-posting__details"})
            job_description_text = job_description_html.find(
                "div", {"class": "show-more-less-html__markup"}).get_text().strip()
        except:
            print(f"Error skipping because not job description found: {job_id}")
            continue
        job_post['job_description_html'] = job_description_html
        job_post['job_description_text'] = job_description_text
        job_post['job_id'] = job_id
        job_post['keyword'] = jobs['keyword']
        job_post_list.append(job_post)
print(f"Total jobs fetched: {len(job_post_list)}")
job_list_df = pd.DataFrame(job_post_list)
if len(job_list_df) > 0:
    # Create a folder for each keyword
    prefix = '../samples/job_listings/'
    for keyword in job_list_df['keyword'].unique():
        folder_path = f"{prefix}{keyword}_jobs"
        os.makedirs(folder_path, exist_ok=True)

    # Save job descriptions as PDF files
    for index, row in job_list_df.iterrows():
        keyword = row['keyword']
        job_id = row['job_id']
        job_position_name = row['position_name']
        job_description_text = row['job_description_text']
        
        # Create a PDF object
        pdf = FPDF()
        
        # Add a page
        pdf.add_page()
        
        # Set font and size
        pdf.set_font("Arial", size=12)
        job_description_text = re.sub(r'[^\x00-\x7F]+', "'", job_description_text) 
        job_position_name = re.sub(r'[^\x00-\x7F]+', "'", job_position_name) 
        # Remove special characters from the job position name
        job_position_name = re.sub(r'[^\w\s]', '', job_position_name)
        # Add job description text to the PDF
        pdf.multi_cell(0, 10, job_description_text)
        
        # Save the PDF file in the corresponding folder
        short_desc = job_description_text[:20].replace(" ", "_")
        pdf_file_path = f"{prefix}{keyword}_jobs/{job_id}_{job_position_name}.pdf"
        pdf.output(pdf_file_path, 'F')
    
    
    job_list_df.to_csv(
    f"location: {location}, date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.csv", index=False)

Page 1 has 133 jobs
Fetching jobs, start: 25 for url: https://www.linkedin.com/jobs/search?keywords=engineer&location=US&start=25
job_id_list length: 7
Page 1 has 122 jobs
Fetching jobs, start: 50 for url: https://www.linkedin.com/jobs/search?keywords=healthcare&location=US&start=50
job_id_list length: 67
Page 1 has 186 jobs
Fetching jobs, start: 75 for url: https://www.linkedin.com/jobs/search?keywords=finance&location=US&start=75
job_id_list length: 127
job id list: [{'job_id': '4174674185', 'keyword': 'engineer'}, {'job_id': '4172312082', 'keyword': 'engineer'}, {'job_id': '4174616956', 'keyword': 'engineer'}, {'job_id': '4174454305', 'keyword': 'engineer'}, {'job_id': '4174601733', 'keyword': 'engineer'}, {'job_id': '4179518973', 'keyword': 'engineer'}, {'job_id': '4175753313', 'keyword': 'engineer'}, {'job_id': '4181143622', 'keyword': 'healthcare'}, {'job_id': '4176876536', 'keyword': 'healthcare'}, {'job_id': '4181025180', 'keyword': 'healthcare'}, {'job_id': '4179338835', 'keyw

/tmp/ipykernel_13202/755128409.py:137: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
/tmp/ipykernel_13202/755128409.py:148: DeprecationWarning: "dest" parameter is deprecated since v2.2.0, unused and will soon be removed
  pdf.output(pdf_file_path, 'F')
